In [ ]:
import requests
import zipfile
import io
import pandas as pd
import numpy as np
!pip install osmnx
import osmnx as ox
from sklearn.neighbors import KDTree
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import folium

In [ ]:
class BikeLaneInfo:
    def __init__(self, u, v, x1, y1, x2, y2, color='green'):
        self.u = u
        self.v = v
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.length = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
        self.accidents = 0
        self.density = 0
        self.color = color

def get_barcelona_data_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        alternate_links = soup.find_all('link', rel='alternate')

        for link in alternate_links:
            city_name = urlparse(link['href']).path.split('/')[-1].split('.')[0]
            if city_name == 'barcelona':
                print(f"Data available for: {city_name}")
                return link['href']

        print("Barcelona data not found.")
        return None
    else:
        print("Failed to access URL.")
        return None

def get_accident_data(city_url, chosen_city):
    response = requests.get(city_url)
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            csv_filename = f"cycling_safety_{chosen_city}.csv"

            for filename in zip_ref.namelist():
                if filename.endswith(csv_filename):
                    with zip_ref.open(filename) as csv_file:
                        try:
                            df = pd.read_csv(csv_file, delimiter=',', encoding='utf-8', low_memory=False)
                            lat_col = [col for col in df.columns if 'latitude' in col.lower()]
                            lon_col = [col for col in df.columns if 'longitude' in col.lower()]
                            if lat_col and lon_col:
                                return df[[lat_col[0], lon_col[0]]].rename(columns={lat_col[0]: 'Latitude', lon_col[0]: 'Longitude'})
                            else:
                                print("Latitude and Longitude columns not found in CSV file.")
                                return None
                        except pd.errors.ParserError as e:
                            print(f"Error reading CSV file: {e}")
                            return None
                    break
            else:
                print("CSV file not found for the chosen city.")
                return None
    else:
        print(f"Failed to access city URL: {city_url}")
        return None

def point_to_line_distance(x1, y1, x2, y2, px, py):
    """Calculates the distance from point (px, py) to the line segment (x1, y1) - (x2, y2)"""
    line_mag = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    if line_mag < 1e-10:
        return float('inf')

    u = ((px - x1) * (x2 - x1) + (py - y1) * (y2 - y1)) / line_mag**2
    if u < 0:
        closest_point = (x1, y1)
    elif u > 1:
        closest_point = (x2, y2)
    else:
        closest_point = (x1 + u * (x2 - x1), y1 + u * (y2 - y1))

    distance = np.sqrt((px - closest_point[0])**2 + (py - closest_point[1])**2)
    return distance

def plot_bike_lanes_with_accidents(city_name, city_url):
    try:
        G = ox.graph_from_place(city_name, network_type='all')

        bike_lanes = []
        for u, v, k, data in G.edges(keys=True, data=True):
            if data.get('highway') == 'cycleway':
                x1, y1 = G.nodes[u]['x'], G.nodes[u]['y']
                x2, y2 = G.nodes[v]['x'], G.nodes[v]['y']
                bike_lane = BikeLaneInfo(u, v, x1, y1, x2, y2)
                bike_lanes.append(bike_lane)

        accident_data = get_accident_data(city_url, city_name)
        if accident_data is not None:
            for accident_index, accident in accident_data.iterrows():
                accident_coords = (accident['Longitude'], accident['Latitude'])
                distances = [point_to_line_distance(bl.x1, bl.y1, bl.x2, bl.y2, accident_coords[0], accident_coords[1]) for bl in bike_lanes]

                min_distance = min(distances)
                if min_distance <= 0.2:  # 500 meters
                    for i, distance in enumerate(distances):
                        if distance == min_distance:
                            bike_lanes[i].accidents += 1

            # Calcular a média de acidentes, ignorando ciclovias com zero acidentes
            accident_counts = [bl.accidents for bl in bike_lanes if bl.accidents > 0]
            if accident_counts:
                mean_accidents = np.mean(accident_counts)
            else:
                mean_accidents = 0
            print(mean_accidents)
            # Calcular a porcentagem de acidentes para cada ciclovia
            for bl in bike_lanes:
                if mean_accidents > 0:
                    bl.density = (bl.accidents / (mean_accidents*2)) * 100
                else:
                    bl.density = 0

            # Aplicar cores com base nas porcentagens
            for bl in bike_lanes:
                if bl.density < 33:
                    bl.color = 'green'
                elif bl.density < 66:
                    bl.color = 'yellow'
                else:
                    bl.color = 'red'

            # Contador do número de ciclovias e das densidades calculadas
            num_bike_lanes = len(bike_lanes)
            num_densities_calculated = len([bl for bl in bike_lanes if bl.density > 0])

            print(f"Total number of bike lanes: {num_bike_lanes}")
            print(f"Number of bike lanes with calculated densities: {num_densities_calculated}")

            # Criar mapa interativo
            m = folium.Map(location=[41.3851, 2.1734], zoom_start=13)

            # Plotar ciclovias no mapa interativo
            for bl in bike_lanes:
                folium.PolyLine(
                    locations=[(bl.y1, bl.x1), (bl.y2, bl.x2)],
                    color=bl.color,
                    weight=5,
                    opacity=0.7
                ).add_to(m)

            # Adicionar marcadores de acidentes
            for accident_index, accident in accident_data.iterrows():
                folium.CircleMarker(
                    location=[accident['Latitude'], accident['Longitude']],
                    radius=3,
                    color='blue',
                    fill=True,
                    fill_color='blue'
                ).add_to(m)

            # Salvar o mapa em um arquivo HTML
            m.save('barcelona_bike_lanes.html')
            print("Interactive map has been saved as 'barcelona_bike_lanes.html'.")
        else:
            print("No accident data available.")
    except Exception as e:
        print("Error occurred:", e)

if __name__ == "__main__":
    url = "https://zenodo.org/record/5603036"
    barcelona_url = get_barcelona_data_url(url)
    if barcelona_url:
        plot_bike_lanes_with_accidents("barcelona", barcelona_url)
    else:
        print("No data available for Barcelona.")